# Intro sulla model selection

Durante l'ultima lezione, abbiamo parlato dell'utilità del validation set per fare l'early stopping. La scelta del "modello migliore" durante il training, valutando la sua performance sul validation set, è un punto chiave della __model selection__.

Abbiamo specificato che ci sono due punti in particolare in cui facciamo la model selection:

1. Rispetto a diverse configurazioni del modello;
2. Data una configurazione, nella fase del training in cui il modello si "comporta meglio" sui dati di validazione.

La scorsa volta abbiamo visto come soddisfare il secondo punto della model selection, oggi vedremo il primo.

# Hyperparameter tuning con Keras

Nelle prime due lezioni, i modelli che abbiamo utilizzato erano sempre dei modelli con delle configurazioni scelte "a caso". Purtroppo, quando creiamo un modello per fare predizioni su nuovi dati, la scelta della configurazione non è una cosa per niente scontata.

Ogni modello è dotato di due tipologie di parametri:
1. I parametri adattivi, ovvero i pesi della rete neurale, che vengono __adattati durante il training__;
2. Gli __iperparametri__, che specificano l'architettura del modello (per esempio, quanti layer deve avere la rete neurale, quante unità per layer), la regolarizzazione, il learning rate, il batch size etc...

I secondi sono fissati __prima del training__, e possono rappresentare un fattore determinante nella qualità del modello (con un modello con troppi layer e unità potremmo avere overfitting, nel caso contrario underfitting) e nella qualità del training (un learning rate troppo alto fa divergere il modello, mentre uno troppo basso rallenta troppo il training e potrebbe restituire soluzioni sub-ottime).

Al giorno d'oggi, purtroppo, non esiste ancora una "*formula magica*" grazie alla quale troviamo la configurazione perfetta. Esistono algoritmi che ottimizzano questa scelta, ma non sono materiale di questo corso.

Quindi, ogni buon ML engineer si affida, generalmente, per il 20% all'esperienza e per il rimanente 80%... al caso, ma con "*tante chances*". :)

## Keras Tuner

[Keras Tuner](https://keras.io/keras_tuner/) è la libreria di alto livello che ci viene offerta da Keras per fare una ricerca di iperparametri.

Questa si basa su due componenti principali:

1. Uno __spazio di ricerca__, che denota tutte le possibili configurazioni di iperparametri da cui possiamo scegliere;
2. Un __algoritmo di ricerca__, che ci dice come dobbiamo muoverci per arrivare al nostro modello ottimo.

In [ ]:
from tensorflow import keras as K
import keras_tuner as kt

In [ ]:
kt.__version__

### Search space API

La API dello spazio di ricerca offre diverse opzioni per scegliere iperparametri sia categorici che numerici (interi e float).

In [ ]:
from keras_tuner import HyperParameters

Creiamo il nostro spazio di ricerca vuoto chiamando la classe `HyperParameters`

In [ ]:
hp = HyperParameters()

`khp.Choice` è il candidato ideale per scegliere da un insieme di parametri categorici. Se, per esempio, dovessimo scegliere fra due attivazioni (`tanh` o `relu`), potremmo utilizzare questa funzione per "scegliere" fra i due.

In [ ]:
help(hp.Choice)
print("Valore di attivazione scelto =", hp.Choice(name='activation', values=['tanh', 'relu']))

`hp.Boolean` quando dobbiamo scegliere qualcosa che sia condizionato da un valore di verità. Un esempio può essere se vogliamo utilizzare il bias o meno sull'ultimo layer (`use_bias=True/False`).

In [ ]:
help(hp.Boolean)
print("Valore booleano =", hp.Boolean(name='use_bias'))

`hp.Fixed` lo utilizziamo quando vogliamo fissare un parametro. Per esempio, vogliamo che il valore di batch_size sia uguale a 32.

In [ ]:
help(hp.Fixed)
print("Valore di batch size =", hp.Fixed(name='batch_size', value=32))

`hp.Float` lo possiamo utilizzare quando vogliamo scegliere valori numerici all'interno di un certo intervallo. Vari esempi di utilizzo possono essere il learning rate, la regolarizzazione e altro ancora.

In [ ]:
help(hp.Float)
print("Valore del learning rate =", hp.Float(name='learning_rate', min_value=0.0001, max_value=0.1, sampling='log'))

`hp.Int` restituisce un valore intero all'interno di un range prestabilito. Può essere utile per scegliere il numero di layers o il numero di unità della nostra rete neurale.

In [ ]:
help(hp.Int)
print("Il numero di layers della rete è", hp.Int(name='units', min_value=1, max_value=5))

Ora che abbiamo il nostro spazio di ricerca, ovvero `hp`, possiamo anche reperire i valori correnti di ogni elemento stabilito nello spazio di ricerca. Questo ci permette di prendere una configurazione del nostro modello.

In [ ]:
print(hp.get('units'))
print(hp.get('learning_rate'))
print(hp.get('batch_size'))
print(hp.get('use_bias'))
print(hp.get('activation'))

Tutto questo, in realtà, viene fatto in automatico dentro la funzione di "creazione del modello", a breve vediamo come.

### Random Search

Come dicevamo poco fa, un ML engineer fa affidamento per il 20% all'esperienza, e per l'80% a caso. In senso letterale.

Con la __Random Search__ definiamo uno spazio di ricerca con un minimo di esperienza (se sappiamo di avere un dataset giocattolo, non utilizzeremo una rete neurale con 50 layers) e creando configurazioni a caso da quello spazio di ricerca.

Keras Tuner offre la classe `RandomSearch`, a cui dobbiamo passare:

1. Una funzione di __creazione del modello rispetto allo spazio di ricerca__;
2. Una funzione obiettivo con cui valuteremo il modello (esempio, loss sul validation set);
3. Un numero di tentativi, ovvero di configurazioni che vogliamo provare.

La funzione `build_model`, quando utilizzata dentro la random search, ci restituirà configurazioni casuali del modello rispetto allo spazio di ricerca che abbiamo definito.

In [ ]:
def build_model(hp):
    model = K.Sequential()
    model.add(K.layers.Flatten())
    model.add(K.layers.Rescaling(scale=1/255.))

    model.add(K.layers.Dense(
        units=hp.Choice('units', [50, 100, 1000]),
        activation='relu'))
    model.add(K.layers.Dense(10, activation='softmax', use_bias=hp.Boolean('use_bias')))

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

Ora creiamo il la nostra `RandomSearch` rispetto al modello fissato, dove generiamo 5 configurazioni casuali. 

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5
)

E vediamo cosa succede con il nostro caro MNIST. :)

In [ ]:
import tensorflow.keras.datasets as kds
from sklearn.model_selection import train_test_split

In [ ]:
(train_X, train_y), (test_X, test_y) = kds.mnist.load_data(path='ds')

In [ ]:
train_X, eval_X, train_y, eval_y = train_test_split(
    train_X, train_y,
    test_size=0.15,
    shuffle=True,
    stratify=train_y
)

La funzione search espone esattamente gli stessi parametri di `model.fit`.

In [ ]:
tuner.search(train_X, train_y, epochs=5, batch_size=1000, validation_data=(eval_X, eval_y))

In [ ]:
best_model = tuner.get_best_models()[0]
prediction = best_model(train_X[:10])
best_model.summary()

In [ ]:
tuner.get_best_hyperparameters()[0].get_config()['values']

# Your Turn!

Prendete il lavoro che avete fatto la scorsa volta sul dataset Boston housing e lanciate una RandomSearch esplorando come segue:
1. Sul primo layer, esplorate sui parametri `units` e `regularization`, e l'attivazione è `tanh`;
2. Il secondo layer è fissato a 100 neuroni, con attivazione `tanh`;
3. Il terzo layer ha un solo neurone senza attivazione.

In [ ]:
(h_train_X, h_train_y), (h_test_X, h_test_y) = kds.boston_housing.load_data()

In [ ]:
import random
import numpy as np

h_train = np.concatenate([h_train_X, h_train_y[:, np.newaxis]], axis=1)
random.seed(42)
random.shuffle(h_train)

h_train, h_eval = h_train[75:], h_train[:75]
h_train_X, h_train_y = h_train[:, :-1], h_train[:, -1]
h_eval_X, h_eval_y = h_eval[:, :-1], h_eval[:, -1]

In [ ]:
(h_train_X.shape, h_train_y.shape), (h_eval_X.shape, h_eval_y.shape)